In [150]:
import pandas as pd
import numpy as np
import nltk, datetime

In [174]:
import lexnlp

ModuleNotFoundError: No module named 'lexnlp'

In [101]:
pd.set_option('display.max_columns', 999)

In [173]:
recalls = pd.read_csv('recalls.csv', 
                      dtype={'RecallID':'Int64', 'RecallNumber': str, 'RecallDate': str,
                             'Description': str, 'URL': str, 'Title': str, 'ConsumerContact': str,
                             'LastPublishDate': str, 'Images': 'object', 'SoldAtLabel': str,
                             'Distributors_CompanyID': 'Int64', 'DistributorsSName': str, 
                             'Hazards_HazardType': str, 'Hazards_HazardTypeID': str, 'Hazards_Name': str,
                             'Importers_CompanyID': 'Int64', 'Importers_Name': str, 'Inconjunctions_URL': str,
                             'Injuries_Name': str, 'ManufacturerCountries_Country': str, 
                             'Manufacturers_CompanyID': 'Int64', 'Manufacturers_Name': str, 'ProductUPCs_UPC': str,
                             'Products_CategoryID': 'Int64', 'Products_Description': str, 'Products_Model': str,
                             'Products_Name': str, 'Products_NumberOfUnits': str, 'Products_Type': str,
                             'Remedies_Name': str, 'RemedyOptions_Option': str, 'Retailers_CompanyID': str,
                             'Retailers_Name': str})

#UPCs loaded as strings; clean for whitespace & non-numeric chars & cast to int
#convert number of units from string to int (remove "About ")

In [164]:
set(recalls['Retailers_CompanyID'][~pd.isna(recalls['Retailers_CompanyID'])])

{'0'}

In [119]:
titles = recalls['Title'].str.split('Recall[s]|Due to', expand=True)
titles = titles.rename(columns={0: 'CompanyShortname', 1: 'ProductsShortname', 2: 'HazardAlt'})
recalls = pd.concat([recalls, titles], axis=1)

In [120]:
recalls.head()

,RecallID,RecallNumber,RecallDate,Description,URL,Title,ConsumerContact,LastPublishDate,Images,SoldAtLabel,Distributors_CompanyID,Distributors_Name,Hazards_HazardType,Hazards_HazardTypeID,Hazards_Name,Importers_CompanyID,Importers_Name,Inconjunctions_URL,Injuries_Name,ManufacturerCountries_Country,Manufacturers_CompanyID,Manufacturers_Name,ProductUPCs_UPC,Products_CategoryID,Products_Description,Products_Model,Products_Name,Products_NumberOfUnits,Products_Type,Remedies_Name,RemedyOptions_Option,Retailers_CompanyID,Retailers_Name,CompanyShortname,ProductsShortname,HazardAlt
0,8726,20056,2020-01-14T00:00:00,This recall involves Thompson’s WaterSeal wate...,https://www.cpsc.gov/Recalls/2020/The-Thompson...,The Thompson’s Company Recalls Aerosol Waterpr...,The Thompson’s Company toll-free at 888-304-37...,2020-01-14T00:00:00,[{'URL': 'https://www.cpsc.gov/s3fs-public/Cap...,NaN,NaN,NaN,NaN,NaN,The contents of the cans can react with the pa...,NaN,NaN,NaN,The Thompson’s Company received approximately ...,United States,NaN,"The Thompson’s Company, of Cleveland, Ohio",NaN,NaN,NaN,NaN,Thompson’s® WaterSeal® Waterproofing Wood Prot...,"About 852,000",NaN,Consumers should immediately stop using Thomps...,Refund,NaN,"Lowe’s Home Improvement, Walmart, Menards, Hom...",The Thompson’s Company,Aerosol Waterproofing Wood and Masonry Protec...,Fire Hazard
1,8723,20050,2020-01-09T00:00:00,This recall involves Holiday Travel mugs with ...,https://www.cpsc.gov/Recalls/2020/Boston-Wareh...,Boston Warehouse Trading Corp. Recalls Holiday...,Boston Warehouse Trading Corp. toll-free at 88...,2020-01-09T00:00:00,[{'URL': 'https://www.cpsc.gov/s3fs-public/1_4...,NaN,NaN,NaN,NaN,NaN,The mugs are mislabeled as microwave safe. If ...,NaN,"Meijer Distribution Inc., of Grand Rapids, Mich.",NaN,The firm has received one report of sparks whe...,China,NaN,"Boston Warehouse Trading Corp., of Norwood, Mass.",NaN,NaN,NaN,NaN,Holiday Travel Mugs,"About 2,400",NaN,Consumers should immediately stop using the re...,Refund,NaN,Exclusively at Meijer stores nationwide from O...,Boston Warehouse Trading Corp.,Holiday Travel Mugs,Fire Hazard; Sold Exclusively at Meijer Stores
2,8724,20051,2020-01-09T00:00:00,This recall involves the Libbey Glass 33.5 oz....,https://www.cpsc.gov/Recalls/2020/Libbey-Glass...,Libbey Glass Recalls Milk Bottles Due to Lacer...,Libbey Glass at 800-982-7063 between 8 a.m. an...,2020-01-09T00:00:00,[{'URL': 'https://www.cpsc.gov/s3fs-public/1_4...,NaN,NaN,NaN,NaN,NaN,"The bottles can break unexpectedly during use,...",NaN,"Libbey Glass Inc., of Toledo, Ohio",NaN,None Reported,China,NaN,NaN,NaN,NaN,NaN,NaN,33.5 oz. Milk Bottles,"About 44,300",NaN,Foodservice establishments and customers shoul...,Refund,NaN,Libby sold the recalled bottles to various foo...,Libbey Glass,Milk Bottles,Laceration Hazard
3,8725,20712,2020-01-09T00:00:00,This recall involves Little Bambino 4 in 1 can...,https://www.cpsc.gov/Recalls/2020/Thesaurus-Gl...,Thesaurus Global Marketing Recalls Tricycles D...,Little Bambino toll-free at 866-633-8202 from ...,2020-01-09T00:00:00,[{'URL': 'https://www.cpsc.gov/s3fs-public/1_4...,NaN,NaN,NaN,NaN,NaN,Paint on the canopy’s frame contains levels of...,NaN,"Thesaurus Global Marketing Inc., of Doral, FL",NaN,None reported,China,NaN,NaN,NaN,NaN,NaN,NaN,Little Bambino 4 in 1 canopy children’s tricycles,About 370,NaN,Consumers should immediately stop using the re...,Refund,NaN,Amazon.com from October 2018 through June 2019...,Thesaurus Global Marketing,Tricycles,Violation of the Federal Lead Paint Ban; Risk...
4,8721,20046,2019-12-20T00:00:00,This recall involves Buderus brand GB125-35 oi...,https://www.cpsc.gov/Recalls/2020/Bosch-Thermo...,Bosch Thermotechnology Recalls Buderus Boilers...,Bosch Thermotechnology at 800-323-1943 from 8 ...,2019-12-20T00:00:00,[{'URL': 'https://www.cpsc.gov/s3fs-public/Scr...,NaN,NaN,"Bosch Thermotechnology Corp., of Watertown, Mass.",NaN,NaN,"The siphon can become blocked, leading to a de...",NaN,"Bosch Ther

In [112]:
recalls['Description'].head(20)

0     This recall involves Thompson’s WaterSeal wate...
1     This recall involves Holiday Travel mugs with ...
2     This recall involves the Libbey Glass 33.5 oz....
3     This recall involves Little Bambino 4 in 1 can...
4     This recall involves Buderus brand GB125-35 oi...
5     This recall involves the frosted balsam soy bl...
6     This recall involves the Burke Merge metal cli...
7     This recall involves all model year 2018 and 2...
8     This recall includes a fuchsia colored Light-U...
9     This recall includes Chadwick and Bailey five-...
10    This recall involves Crewcuts boys’ denim pant...
11    This recall involves Profile Design Aeria Ulti...
12    The recall involves four and five burner model...
13    This recall involves all Surly 8-Pack Racks an...
14    This recall involves all model year 2017, 2018...
15    This recall involves StoveTop FireStop Rangeho...
16    This recall involves wooden DaVinci Bailey Bas...
17    This recall involves the STANLEY brand 16 

In [115]:
reports = pd.read_csv('SPDB/IncidentReports.csv', encoding="ISO-8859-1", dtype='object')

In [117]:
reports.columns

Index(['Report No.', 'Report Date',
       'Sent to Manufacturer / Importer / Private Labeler', 'Publication Date',
       'Category of Submitter', 'Product Description', 'Product Category',
       'Product Sub Category', 'Product Type', 'Product Code',
       'Manufacturer / Importer / Private Labeler Name', 'Brand',
       'Model Name or Number', 'Serial Number', 'UPC', 'Date Manufactured',
       'Manufacturer Date Code', 'Retailer', 'Retailer State', 'Purchase Date',
       'Purchase Date Is Estimate', 'Incident Description', 'City', 'State',
       'ZIP', 'Location', '(Primary) Victim Severity',
       '(Primary) Victim's Gender', 'My Relation To The (Primary) Victim',
       '(Primary) Victim's Age (years)', 'Submitter Has Product',
       'Product Was Damaged Before Incident', 'Damage Description',
       'Damage Repaired', 'Product Was Modified Before Incident',
       'Have You Contacted The Manufacturer', 'If Not Do You Plan To',
       'Answer Explanation', 'Company Comments

In [163]:
set(recalls['UPC'])

{-1,
 '0 7789028735 4',
 '000014424410',
 '000346327526',
 '0004266643392',
 '0040818220207',
 '010121400054',
 '011546208270',
 '012480011328',
 '012914058022',
 '015381142201',
 '015715010145',
 '016438100267',
 '016438897204',
 '017845000591',
 '018285004224',
 '018866001024',
 '022253206152',
 '022253208163',
 '024131114849',
 '024838170001',
 '024911',
 '026669082522',
 '027557111379',
 '029807891864',
 '031412673100',
 '03156811016',
 '033572100014',
 '035203000380',
 '037434455083',
 '037916889603',
 '040388011137',
 '041163270060',
 '041220843473',
 '043377746317',
 '045774700237',
 '047475454198',
 '048419616634',
 '049696604536',
 '050051226409',
 '050276854722',
 '050743109201',
 '05698601008-3',
 '057355271319',
 '059584052301',
 '06301370069',
 '069302970040',
 '072000729700',
 '074590875416',
 '074972010251',
 '075656009165',
 '076001304737',
 '076354995262',
 '076501222733',
 '077802111326',
 '078300010234',
 '07869842221311',
 '080692050101',
 '080692999301',
 '08229431

In [138]:
recalls = recalls.rename(columns={'ProductUPCs_UPC': 'UPC'})
#recalls['UPC'][recalls['UPC'].isna()] = -1
#recalls['UPC'] = recalls['UPC'].astype('int64')
reports['UPC'][reports['UPC'].isna()] = -2
reports['UPC'] = reports['UPC'].astype('int64')

ValueError: invalid literal for int() with base 10: "'3053903870"

In [131]:
match = reports.join(recalls, on='UPC', how='left')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [128]:
recalls.dtypes

RecallID                         object
RecallNumber                     object
RecallDate                       object
Description                      object
URL                              object
Title                            object
ConsumerContact                  object
LastPublishDate                  object
Images                           object
SoldAtLabel                      object
Distributors_CompanyID           object
Distributors_Name                object
Hazards_HazardType               object
Hazards_HazardTypeID             object
Hazards_Name                     object
Importers_CompanyID              object
Importers_Name                   object
Inconjunctions_URL               object
Injuries_Name                    object
ManufacturerCountries_Country    object
Manufacturers_CompanyID          object
Manufacturers_Name               object
UPC                              object
Products_CategoryID              object
Products_Description             object


In [130]:
reports.dtypes

Report No.                                           object
Report Date                                          object
Sent to Manufacturer / Importer / Private Labeler    object
Publication Date                                     object
Category of Submitter                                object
Product Description                                  object
Product Category                                     object
Product Sub Category                                 object
Product Type                                         object
Product Code                                         object
Manufacturer / Importer / Private Labeler Name       object
Brand                                                object
Model Name or Number                                 object
Serial Number                                        object
UPC                                                  object
Date Manufactured                                    object
Manufacturer Date Code                  

In [142]:
len(set(recalls['UPC']))

163

In [157]:
min(pd.to_datetime(reports['Report Date']))

Timestamp('2011-03-11 00:00:00')

In [158]:
sum(pd.to_datetime(recalls['RecallDate']) >= pd.to_datetime('3/11/2011'))

2535

In [ ]:
7766